In [61]:
# Installation tensorflow_text 
!pip install -q -U "tensorflow-text==2.8.*"


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import sys, os
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import glob

import tensorflow as tf
import tensorflow_text as text

Notes qui peuvent aider:
- English-to-French translation
- Bilingual, parallel corpora provided by ACL WMT  14

reduce the size of the corpus using the selection method by Axelrod et al.


Preprocesing of the data:
- tokenization
- use a shortlist of 30,000 most frequent words in each langage to train the model
- any word not included in the shortlist is mapped to a special token ([UNK])
- no other preprocessing method (no lowercasinf or stemming)

- 2 types of data 


sentences of up to 30 words and sentences of up to 50 words

- each update direction is computed using a minibatch of 80 sentences

- once the model is trained, use a beam search to find a translation that approximately maximizes the conditional probability






In [60]:
# Download the file
path_fr = glob.glob('./training/test_fr.txt')
path_en = glob.glob('./training/test_en.txt') 

def load_data(path):
    sentences = []
    with open(path,'r',encoding="utf8") as file:
        for line in file:
            sentences.append(line)

    return np.array(sentences)

source_raw = load_data(path_fr[0])
target_raw = load_data(path_en[0])

#print(type(source_raw))
#print(len(source_raw)==len(target_raw))
#print(source_raw[-1])
#print(target_raw[-1])

# Create a tf.data dataset
BUFFER_SIZE = len(source_raw)
BATCH_SIZE = 64 # 80

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((source_raw[is_train], target_raw[is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((source_raw[~is_train], target_raw[~is_train]))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print()
  print(example_target_strings[:5])
  break

# TEXT PREPROCESSING

# Standardization
def tf_Standardization(text):
  # Split accented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

# Vectorization : tf  layer which will handle the vocabulary extraction and conversion of input text to sequences of tokens.

MAX_VOCAB_SIZE = 3000

context_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_Standardization,
    max_tokens=MAX_VOCAB_SIZE,
    ragged=True)



<class 'numpy.ndarray'>
True
Ma question porte sur un sujet qui est à l'ordre du jour du jeudi et que je soulèverai donc une nouvelle fois.

My question relates to something that will come up on Thursday and which I will then raise again.
tf.Tensor(
[b"Oui, Monsieur Evans, je pense qu'une initiative dans le sens que vous venez de sugg\xc3\xa9rer serait tout \xc3\xa0 fait appropri\xc3\xa9e.\n"
 b"Je voudrais vous demander un conseil au sujet de l'article 143, qui concerne l'irrecevabilit\xc3\xa9.\n"
 b"L'une des personnes qui vient d'\xc3\xaatre assassin\xc3\xa9e au Sri Lanka est M. Kumar Ponnambalam, qui avait rendu visite au Parlement europ\xc3\xa9en il y a quelques mois \xc3\xa0 peine.\n"
 b"Ne pensez-vous pas, Madame la Pr\xc3\xa9sidente, qu'il conviendrait d'\xc3\xa9crire une lettre au pr\xc3\xa9sident du Sri Lanka pour lui communiquer que le Parlement d\xc3\xa9plore les morts violentes, dont celle de M. Ponnambalam, et pour l'inviter instamment \xc3\xa0 faire tout ce qui est en so